In [33]:

from fairgbm import FairGBMClassifier
from aif360.sklearn.metrics import average_odds_difference
from aif360.metrics import ClassificationMetric
from fairlearn.metrics import MetricFrame as MF
from aif360.sklearn.metrics import equal_opportunity_difference
from sklearn.preprocessing import LabelEncoder  
from sklearn.metrics import f1_score, confusion_matrix, make_scorer
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate
from fairlearn.metrics import (
    count,
    selection_rate,
    equalized_odds_difference,
    false_positive_rate,
    false_negative_rate,
    true_negative_rate
)

from fairlearn.datasets import fetch_adult
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier


In [17]:
data = fetch_adult(as_frame=True)
X_raw = data.data
y = (data.target == ">50K") * 1
A = X_raw["sex"]

(X_train, X_test, y_train, y_test, A_train, A_test) = train_test_split(
    X_raw, y, A, test_size=0.3, random_state=12345, stratify=y
)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
A_train = A_train.reset_index(drop=True)
A_test = A_test.reset_index(drop=True)

/home/azucena/miniconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [18]:

numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)

fairgbm_clf = FairGBMClassifier(
    constraint_type=["FNR","FPR"],    # constraint on equal group-wise TPR (equal opportunity)
    n_estimators=200,         # core parameters from vanilla LightGBM
    random_state=42,          # ...
)

pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            fairgbm_clf,
        ),
    ]
)

In [8]:
le = LabelEncoder()
A_train_num = le.fit_transform(A_train)
A_test_num = le.transform(A_test)

In [19]:

# Instantiate
fairgbm_clf = FairGBMClassifier(
    constraint_type=["FNR","FPR"],    # constraint on equal group-wise TPR (equal opportunity)
    n_estimators=200,         # core parameters from vanilla LightGBM
    random_state=42,          # ...
)

# Train using features (X), labels (Y), and sensitive attributes (S)
fairgbm_clf.fit(X_train, y_train, constraint_group=A_train_num)
# NOTE: labels (Y) and sensitive attributes (S) must be in numeric format


FairGBMClassifier(constraint_type=['FNR', 'FPR'], n_estimators=200,
                  random_state=42)

In [20]:
# Predict
Y_test_pred = fairgbm_clf.predict(X_test)

In [21]:
from fairlearn.metrics import equalized_odds_difference
import numpy as np

#def objective_function(fairness_metric, model_metric, y_test, y_pred, sensitive_features, alpha = .5):
fair_value = np.abs(equalized_odds_difference(y_test, Y_test_pred, sensitive_features=A_test))
model_value = f1_score(y_test, Y_test_pred) 

print(fair_value)
print(model_value)

0.052513408389157884
0.7131195335276967


In [25]:
print(y_test.sum())
print(Y_test_pred.sum())

3506
3354


In [27]:
average_odds_difference(y_test,Y_test_pred, prot_attr=A_test, priv_group='Male',pos_label=1)

-0.05164667684821955

In [28]:
equal_opportunity_difference(y_test,Y_test_pred, prot_attr=A_test, priv_group='Male',pos_label=1)

-0.05077994530728125

In [ ]:
ClassificationMetric(y_test,Y_test_pred,privileged_groups= [{'sex': 1}], unprivileged_groups= [{'sex': 0}])

In [37]:


tnr = MF(metrics=true_negative_rate,y_true = y_test, y_pred = Y_test_pred, sensitive_features = A_test)
tnr.by_group[1][0][0]
    
    

IndexError: invalid index to scalar variable.

In [35]:
tnr